In [53]:
!pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
!pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:
!pip install -U langchain_google_genai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [56]:
import os
from dotenv import load_dotenv

In [57]:
load_dotenv()

True

In [58]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [59]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [60]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [61]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [62]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [63]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [64]:
llm.invoke("What is Gemini pro from google generative AI?").content

'Google Generative AI doesn\'t have a product specifically called "Gemini Pro." It\'s possible there\'s been some confusion or a recent name change. \n\nHowever, Google does have powerful offerings in the generative AI space. Here are some possibilities you might be thinking of:\n\n* **Google Gemini:** This is Google\'s latest and most advanced AI model, announced in December 2023. It\'s a multimodal model, meaning it can understand and generate different types of data, including text, code, images, audio, and video. Gemini comes in different sizes:\n    * **Gemini Ultra:** The largest and most capable model, designed for highly complex tasks.\n    * **Gemini Pro:** A smaller, more efficient model suitable for a wide range of tasks.\n    * **Gemini Nano:** The most efficient model, designed for on-device applications.\n* **Generative AI features in Google products:** Google is integrating generative AI into many of its products, including Search, Assistant, Ads, and more. These feature

In [65]:
from langchain_chroma import Chroma

In [66]:
from langchain_community.document_loaders import WebBaseLoader

In [67]:
from langchain_core.output_parsers import StrOutputParser

In [68]:
from langchain_core.runnables import RunnablePassthrough

In [69]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [70]:
import bs4

In [71]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

In [72]:
docs = loader.load()

In [73]:
print(docs[0].page_content)



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

In [74]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [75]:
splits = text_splitter.split_documents(docs)

In [76]:
from langchain.embeddings import HuggingFaceEmbeddings

In [77]:
model_name = "BAAI/bge-small-en-v1.5"

In [78]:
encode_kwargs = {'normalize_embeddings':True}

In [79]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [80]:
embedding_function = HuggingFaceEmbeddings(
    model_name = model_name,
    encode_kwargs = encode_kwargs
)

In [81]:
vector_store = Chroma.from_documents(
            splits,
            embedding_function
             )

In [82]:
retriever = vector_store.as_retriever()

In [83]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7fd824623490>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7fd8246a3160>, default_metadata=())

In [84]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [85]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [86]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [87]:
rag_chain = (
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [88]:
rag_chain.invoke("What is Short-term memory?")

'Short-term memory (STM), or working memory, is the temporary storage of information that is currently being used. It is believed to have a capacity of around 7 items and lasts for approximately 20-30 seconds. STM is essential for complex cognitive tasks like learning and reasoning. \n'

In [89]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [90]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

In [91]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [92]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt,
)

In [93]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [94]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

In [95]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [96]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


In [97]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [98]:
from langchain_core.messages import HumanMessage

In [99]:
chat_history=[]

In [100]:
question1= "What is Task Decomposition?"

In [101]:
response1=rag_chain.invoke({"input":question1,"chat_history": chat_history})

In [102]:
chat_history.extend([HumanMessage(content=question1), response1["answer"]])

In [109]:
from IPython.display import Markdown, display

In [110]:
display(Markdown(response1["answer"]))

Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. This allows large language models (LLMs) to tackle intricate problems more effectively by addressing them in a step-by-step manner.  This approach improves performance and provides insights into the model's reasoning process. 


In [104]:
response1

{'input': 'What is Task Decomposition?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?'),
  "Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. This allows large language models (LLMs) to tackle intricate problems more effectively by addressing them in a step-by-step manner.  This approach improves performance and provides insights into the model's reasoning process. \n"],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompo

In [105]:
question2="what are common ways of doing it?"

In [106]:
response2=rag_chain.invoke({"input":question2,"chat_history": chat_history})

In [107]:
response2

{'input': 'what are common ways of doing it?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?'),
  "Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. This allows large language models (LLMs) to tackle intricate problems more effectively by addressing them in a step-by-step manner.  This approach improves performance and provides insights into the model's reasoning process. \n"],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Finite context length: The restricted context capacity limits the inclusion of historical information, detailed instructions, API call context, and responses. The design of the system has to work with this limited communication bandwidth, while mechanisms like self-reflection to learn from past mistakes would benefit a lot from long or infinite context windows. Although vector stores and retrieval can provide access to a larger

In [111]:
display(Markdown(response2["answer"]))

There are numerous approaches to task decomposition for LLMs, but two common ones include:

1. **Hierarchical decomposition:**  Breaking down a task into a hierarchy of subtasks, much like creating an outline. This helps LLMs understand the overall structure and tackle sub-problems in a logical order. 
2. **Modular decomposition:** Dividing a task into independent modules that can be executed concurrently or in any order. This is particularly useful when parts of a task are unrelated and can be addressed by specialized LLMs. 


In [112]:
question3="for writing a novel what i can do?"

In [113]:
response3=rag_chain.invoke({"input":question3,"chat_history": chat_history})

In [114]:
response3["answer"]

"For writing a novel using task decomposition, you can break down the process into smaller steps:\n\n**1. Brainstorming & Concept:**\n   - **Premise:** Come up with a logline or elevator pitch for your story.\n   - **Characters:** Develop your protagonist, antagonist, and supporting characters.\n   - **Setting:** Decide on the time period, location, and world of your story.\n   - **Plot Outline:** Sketch a rough outline of the major events in your novel.\n\n**2. Detailed Outline:**\n   - **Expand Plot:** Break down your plot outline into chapters or acts.\n   - **Develop Conflicts:** Introduce conflicts and obstacles for your characters.\n   - **Character Arcs:** Plan the development and growth of your characters.\n   - **Subplots (Optional):** Create subplots that intertwine with the main story.\n\n**3. Writing First Draft:**\n   - **Focus on Story:** Write the first draft without worrying too much about perfection.\n   - **Set Daily Goals:** Aim for a specific word count or chapter c

In [115]:
response3

{'input': 'for writing a novel what i can do?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?'),
  "Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. This allows large language models (LLMs) to tackle intricate problems more effectively by addressing them in a step-by-step manner.  This approach improves performance and provides insights into the model's reasoning process. \n"],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by

In [117]:
display(Markdown(response3["answer"]))

For writing a novel using task decomposition, you can break down the process into smaller steps:

**1. Brainstorming & Concept:**
   - **Premise:** Come up with a logline or elevator pitch for your story.
   - **Characters:** Develop your protagonist, antagonist, and supporting characters.
   - **Setting:** Decide on the time period, location, and world of your story.
   - **Plot Outline:** Sketch a rough outline of the major events in your novel.

**2. Detailed Outline:**
   - **Expand Plot:** Break down your plot outline into chapters or acts.
   - **Develop Conflicts:** Introduce conflicts and obstacles for your characters.
   - **Character Arcs:** Plan the development and growth of your characters.
   - **Subplots (Optional):** Create subplots that intertwine with the main story.

**3. Writing First Draft:**
   - **Focus on Story:** Write the first draft without worrying too much about perfection.
   - **Set Daily Goals:** Aim for a specific word count or chapter completion each day.
   - **Don't Edit Yet:** Resist the urge to edit while writing the first draft.

**4. Revision & Editing:**
   - **Self-Edit:** Review your manuscript for plot holes, inconsistencies, and pacing.
   - **Beta Readers:** Share your work with trusted readers for feedback.
   - **Professional Editor (Optional):** Consider hiring an editor for a final polish.

**5. Polishing & Publishing:**
   - **Final Revisions:** Implement feedback from beta readers and editors.
   - **Formatting:** Format your manuscript for print or digital publication.
   - **Cover Design:** Create an eye-catching cover for your book.
   - **Publishing:** Choose a publishing platform or agent to share your work.

Remember, this is just a general guideline. You can adjust the steps and order to fit your own writing process and preferences. 
